In [1]:
from collections import Counter, deque

## Define different groups representing the pocket cube

Define basic moves of the pocket cube as elements of the symmetric group $S_{24}$

In [2]:
U = [(1,2,3,4),(5,17,13,9),(6,18,14,10)]
L = [(5,6,7,8),(1,9,21,19),(4,12,24,18)]
R = [(13,14,15,16),(2,20,22,10),(3,17,23,11)]
F = [(9,10,11,12),(4,13,22,7),(3,16,21,6)]
B = [(17,18,19,20),(1,8,23,14),(2,5,24,15)]
D = [(21,22,23,24),(11,15,19,7),(12,16,20,8)]
x = [(5,8,7,6),(1,19,21,9),(2,20,22,10),(3,17,23,11),(4,18,24,12),(13,14,15,16)] # turn front to top
y = [(1,2,3,4),(5,17,13,9),(6,18,14,10),(7,19,15,11),(8,20,16,12),(21,24,23,22)] # turn right to front
z = [(8,1,14,23),(5,2,15,24),(7,4,13,22),(6,3,16,21),(9,10,11,12),(17,20,19,18)] # turn top to right 

Define group generated by all the basic moves. A move like $L R'$ generates a cube different from the identitiy cube. 

In [3]:
fullCube = PermutationGroup([U,L,R,F,B,D])
print(fullCube)
print(fullCube.order())
print(fullCube.exponent())

Permutation Group with generators [(7,11,15,19)(8,12,16,20)(21,22,23,24), (3,16,21,6)(4,13,22,7)(9,10,11,12), (2,20,22,10)(3,17,23,11)(13,14,15,16), (1,2,3,4)(5,17,13,9)(6,18,14,10), (1,8,23,14)(2,5,24,15)(17,18,19,20), (1,9,21,19)(4,12,24,18)(5,6,7,8)]
88179840
2520


Define group of full rotations of the cube in space.

In [4]:
rotations = PermutationGroup([x,y,z])
print(rotations)
print(rotations.order())
print(rotations.exponent())

Permutation Group with generators [(1,2,3,4)(5,17,13,9)(6,18,14,10)(7,19,15,11)(8,20,16,12)(21,24,23,22), (1,14,23,8)(2,15,24,5)(3,16,21,6)(4,13,22,7)(9,10,11,12)(17,20,19,18), (1,19,21,9)(2,20,22,10)(3,17,23,11)(4,18,24,12)(5,8,7,6)(13,14,15,16)]
24
12


Define pocket cube where the cubie on the bottom left back is fixed. This is equivalent to the cube fixed in space. The operation $L$ would be equivalent to $R'$.

In [9]:
cube = PermutationGroup([U,R,F]); # cube where bottom left back cubie is fixed
print(cube)
print(cube.order())
print(cube.exponent())

Permutation Group with generators [(3,16,21,6)(4,13,22,7)(9,10,11,12), (2,20,22,10)(3,17,23,11)(13,14,15,16), (1,2,3,4)(5,17,13,9)(6,18,14,10)]
3674160
1260


## Enumerate all combinations of the pocket cube using breath first search

Define notation used to denote basic moves

In [10]:
Uc = cube(U)
Rc = cube(R)
Fc = cube(F)
move_notation = {Uc : "U", Uc^2 : "U2", Uc.inverse() : "U'", 
                 Rc : "R", Rc^2 : "R2", Rc.inverse() : "R'", 
                 Fc : "F", Fc^2 : "F2", Fc.inverse() : "F'"}

For the pocket cube, computes all distinct positions of the cubes which can be reached in ``max_length`` steps using the quarter turn metric.

In [12]:
def get_factorizations_QTM(max_length = 100) :
    
    all_moves = [Uc, Rc, Fc, Uc.inverse(), Rc.inverse(), Fc.inverse()] # quarter turn metric
    
    all_factorizations = dict()
    cubes_to_consider = deque()
    cubes_already_considered = set()
    cubes_to_consider.append([cube.identity(), "", 0, cube.identity()])
    cubes_already_considered.add(cube.identity())
    
    while cubes_to_consider :
        new_cube, notation, length, last_move = cubes_to_consider.popleft()
        all_factorizations[new_cube] = (notation, length)
        
        # make sure that we only do moves on current cube that do not
        # undo the last move 
        useful_moves = list(set(all_moves) - set([last_move.inverse()]))

        # do all useful moves on current cube and add it to queue if it has not been
        # considered yet
        for move in useful_moves :
            moved_cube = new_cube*move
            if moved_cube not in cubes_already_considered and length + 1 <= max_length :
                # found new cube
                cubes_to_consider.append([moved_cube, 
                                          notation + move_notation[move], 
                                          length + 1, 
                                          move])
                cubes_already_considered.add(moved_cube)
    
    return all_factorizations

For the pocket cube, computes all distinct positions of the cubes which can be reached in ``max_length`` steps using the half turn metric.

In [11]:
def get_factorizations_HTM(max_length = 100) :
    
    all_moves = [Uc, Uc^2, Rc, Rc^2, Fc, Fc^2, Uc.inverse(), Rc.inverse(), Fc.inverse()] # half turn metric
    URF_moves = [[Uc, Uc.inverse(), Uc^2], [Rc, Rc.inverse(), Rc^2], [Fc, Fc.inverse(), Fc^2]]
    
    all_factorizations = dict()
    cubes_to_consider = deque()
    cubes_already_considered = set()
    cubes_to_consider.append([cube.identity(), "", 0, 0])
    cubes_already_considered.add(cube.identity())
    
    while cubes_to_consider :
        new_cube, notation, length, last_move = cubes_to_consider.popleft()
        all_factorizations[new_cube] = (notation, length)
        
        # make sure that we only do moves on current cube that do not
        # undo the last move or could have been done together with the previous move
        useful_moves = all_moves
        for moves_sides in URF_moves :
            if last_move in moves_sides :
                useful_moves = list(set(all_moves) - set(moves_sides))
                break

        # do all useful moves on current cube and add it to queue if it has not been
        # considered yet
        for move in useful_moves :
            moved_cube = new_cube*move
            if moved_cube not in cubes_already_considered and length + 1 <= max_length :
                # found new cube
                cubes_to_consider.append([moved_cube, 
                                          notation + move_notation[move], 
                                          length + 1, 
                                          move])
                cubes_already_considered.add(moved_cube)
    
    return all_factorizations

Generate a dictionary of all possible pocket cube positions and how they can be reached. Print the minimal number of moves necessary to reach certain positions.

In [13]:
factorizations_HTM = get_factorizations_HTM()
print(len(factorizations_HTM))
move_lengths = [factorization[1] for factorization in factorizations_HTM.values()]
Counter(move_lengths)

3674160


Counter({9: 1887748, 8: 870072, 10: 623800, 7: 227536, 6: 50136, 5: 9992, 11: 2644, 4: 1847, 3: 321, 2: 54, 1: 9, 0: 1})

In [14]:
factorizations_QTM = get_factorizations_QTM(100)
print(len(factorizations_QTM))
move_lengths_QTM = [factorization[1] for factorization in factorizations_QTM.values()]
Counter(move_lengths_QTM)

3674160


Counter({11: 1350852, 10: 930588, 12: 782536, 9: 360508, 8: 114149, 13: 90280, 7: 33058, 6: 8969, 5: 2256, 4: 534, 14: 276, 3: 120, 2: 27, 1: 6, 0: 1})

In [14]:
factorizations_HTM[cube([(2,17,14),(3,10,13)])] # turn top right cubies in place

("UR2F2U'R'UF'RU'R", 10)

## Finding move sequences with different orders

In [27]:
# given a dictionary of all possible positions of the cube, the method returns
# 1. a dictionary of cube positions of the form order : (string_rep, length)
#    where order is the order of the element, string_rep is a string representing
#    the element and length the number of moves used in the position (which is minimal)
#    among the positions of this order
# 2. a counter object counting how many positions have certain orders
def get_smallest_orders(factorization) :
    orders = dict()
    all_orders = []
    for elem in cube :
        order = elem.order()
        all_orders.append(order)
        factor = factorization[elem]
        if order not in orders :
            orders[order] = factor
        elif factor[1] < orders[order][1] :
            orders[order] = factor
    return orders, Counter(all_orders)

In [16]:
smallest_orders, orders_counted = get_smallest_orders(factorizations_HTM)
print(orders_counted)

Counter({18: 714420, 12: 657720, 7: 524880, 6: 521766, 15: 326592, 30: 244944, 9: 215460, 36: 204120, 10: 122472, 4: 56700, 5: 40824, 3: 40418, 2: 3843, 1: 1})


We can see that the element with biggest order has order 36 (and there are 204120 elements with this order). We can now print elements of all possible orders. 

In [28]:
for i in range(1,50) :
    if i in smallest_orders :
        print(f"Order = {i}, {smallest_orders[i]}")

Order = 1, ('', 0)
Order = 2, ('U2', 1)
Order = 3, ('U2F2', 2)
Order = 4, ("U'", 1)
Order = 5, ("R'F'U'F", 4)
Order = 6, ("U'F2", 2)
Order = 7, ("U'FR2", 3)
Order = 9, ("U'F", 2)
Order = 10, ("R'U2F2", 3)
Order = 12, ("R'U'F'", 3)
Order = 15, ("U'F'", 2)
Order = 18, ("U'FR'", 3)
Order = 30, ("R'F'U", 3)
Order = 36, ("R'F2U'F", 4)
